# Kaggle Instacart Classification
This notebook contains the code for classifying whether or not an item will be in a user's most recent cart in the Kaggle Instacart Contest dataset. Most of the functionality (SQL queries, feature engineering, and model training) lives in **build_models.py**, which can be run through a notebook or the command line. If the script is run on a cloud instance, the contents of the cloud **data** directory can be copied into the local version in order to be loaded by this notebook.

### Table of Contents
* [1. Imports and Functions](#sec1)
* [2. Train Models or Load Models From Files](#sec2)
* [3. Feature Engineering](#sec3)
* [4. Function for Probability Threshold Adjustment](#sec4)
* [5. Per-Item Model Evaluation](#sec5)
* [6. Mean Per-User Model Evaluation](#sec6)
* [7. Finding the Most Influential Features](#sec7)
* [8. Histograms of User F1 Scores](#sec8)
* [9. Sample User Cart Classification](#sec9)

<a id='sec1'></a>
### 1. Imports and Functions
* **build_models**: Queries SQL database, engineers features, and trains models
* **read_pickle**: Reads the given pickle file

In [9]:
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

sys.path.append('./code')
import build_models as bm
from lw_pickle import read_pickle

<a id='sec2'></a>
### 2. Train Models or Load Models From Files
I offloaded SQL queries, feature engineering, and model training to the **build_models.py** script in the **code** directory. The script saves a dataframe of testing set data called **test_df** into a .csv file and the logistic regression and gradient boosting decision tree models, **lrm** and **gbm** respectively, into pickle files. All test set and model data is stored in the **data** directory and if they exist, they will be loaded by this notebook. If they do not exist, the notebook trains models for a limited subset of the data (1,000 users) to ensure that the process does not overtax system resources. For final model evaluation, I trained models on the entire dataset of 13,120 users using an AWS EC2 instance and copied the resulting files over to the local **data** directory. My computer had enough RAM to train on 5,000 users in under 10 minutes, but could not handle the full dataset. The 2xLarge EC2 instance took **1 hour and 47 minutes** to train models on the full dataset.

In [10]:
# Setting force_rebuild to True will result in a local re-run of the
# model build script regardless of whether or not local data exists
force_rebuild = False
test_df_csv = 'data/test_df.csv'
model_lrm_pickle = 'data/model_lrm.pk'
model_gbm_pickle = 'data/model_gbm.pk'

# Attempts to load test set and models from local files
test_df = pd.read_csv(test_df_csv, index_col=0)
lrm = read_pickle(model_lrm_pickle)
gbm = read_pickle(model_gbm_pickle)

# Runs build_models script if test set or models did not load
pk_types = [type(x) for x in [test_df, lrm, gbm]]
if type(None) in pk_types or force_rebuild:
    data = bm.build_models(user_limit=1000)
    test_df = data.test_df
    lrm = data.lrm
    gbm = data.gbm
    
# Gets relevant Dataframe columns from build_models
id_col = bm.build_models.id_col
y_col = bm.build_models.y_col
x_cols = bm.build_models.x_cols

<a id='sec3'></a>
### 3. Feature Engineering
All of these features are created in the last step through the **build_models.py** script and used to train both models:
* **percent_in_user_orders**: Percent of a user's orders in which an item appears
* **percent_in_all_orders**: Percent of all orders in which an item appears
* **in_last_cart**: 1 if an item appears in a user's most recent prior order, 0 if not
* **in_last_five**: Number of orders in a user's five most recent prior orders in which an item appears
* **total_user_orders**: Total number of orders placed by a user
* **mean_orders_between**: Average number of orders between which an item appears in a user's order
* **mean_days_between**: Average number of days between which an item appears in a user's order
* **orders_since_newest**: Number of orders between the last user order containing an item and the most recent order
* **days_since_newest**: Number of days between the last user order containing an item and the most recent order
* **product_reorder_proba**: Probability that any user reorders an item
* **user_reorder_proba**: Probability that a user reorders any item
* **mean_cart_size**: Average user cart (aka order) size
* **mean_cart_percentile**: Average percentile of user cart add order for an item
* **mean_hour_of_week**: Average hour of the week that a user orders an item (168 hours in a week)
* **newest_cart_size**: Number of items in the most recent cart
* **newest_hour_of_week**: Hour of the week that the most recent order was placed
* **cart_size_difference**: Absolute value of the difference between the average size of the orders containing an item and the size of the most recent order
* **hour_of_week_difference**: Absolute value of the difference between the average hour of the week in which a user purchases an item and the hour of the week of the most recent order

<a id='sec4'></a>
### 4. Function for Probability Threshold Adjustment
In order to account for the class imbalance that results from most items in a user's order history not being in their current cart, I adjust the probability threshold separating the two classes.

In [12]:
def adjust_threshold(proba, threshold=.5):
    return (proba[:, 1] >= threshold).astype(int)

<a id='sec5'></a>
### 5. Per-Item Model Evaluation
I calculated F1 scores for both the logistic regression and gradient boosting decision tree models treating each item in a user's order history with equal weight. I chose F1 to balance precision and recall and adjusted the probability threshold to maximize F1 scores for each model.

In [14]:
# Calculates logistic regression F1 score using base .5 probability threshold
lrm_proba = lrm.predict_proba(test_df[x_cols])
lrm_pred = adjust_threshold(lrm_proba)
lrm_base_f1 = round(f1_score(test_df[y_col], lrm_pred), 3)

# Calculates logistic regression F1 score using optimized probability threshold
lrm_th = .17
lrm_adj = adjust_threshold(lrm_proba, lrm_th)
lrm_opt_f1 = round(f1_score(test_df[y_col], lrm_adj), 3)

print('Logistic Regression:')
print(f'F1 Score @ 0.50 prob. threshold: {lrm_base_f1}')
print(f'F1 Score @ {lrm_th} prob. threshold: {lrm_opt_f1}')

Logistic Regression:
F1 Score @ 0.50 prob. threshold: 0.313
F1 Score @ 0.17 prob. threshold: 0.447


In [15]:
# Calculates gradient boosting trees F1 score using base .5 probability threshold
gbm_proba = gbm.predict_proba(test_df[x_cols])
gbm_pred = adjust_threshold(gbm_proba)
gbm_base_f1 = round(f1_score(test_df[y_col], gbm_pred), 3)

# Calculates gradient boosting trees F1 score using optimized probability threshold
gbm_th = .21
gbm_adj = adjust_threshold(gbm_proba, gbm_th)
gbm_opt_f1 = round(f1_score(test_df[y_col], gbm_adj), 3)

print('Gradient Boosting Decision Trees:')
print(f'F1 Score @ 0.50 prob. threshold: {gbm_base_f1}')
print(f'F1 Score @ {gbm_th} prob. threshold: {gbm_opt_f1}')

Gradient Boosting Decision Trees:
F1 Score @ 0.50 prob. threshold: 0.338
F1 Score @ 0.21 prob. threshold: 0.461


<a id='sec6'></a>
### 6. Mean Per-User Model Evaluation
I created a separate metric to better evaluate how the models perform for the average user. If either model were incorporated into a recommendation engine the user-based metric would better represent its performance.

In [16]:
lrm_adj = adjust_threshold(lrm_proba, .16)
gbm_adj = adjust_threshold(gbm_proba, .17)

score_df = test_df[[id_col, y_col]].reset_index(drop=True)
score_df['lrm'] = lrm_adj
score_df['gbm'] = gbm_adj
users = score_df[id_col].unique()

lrm_f1 = []
gbm_f1 = []
for user in users:
    user_df = score_df[score_df[id_col] == user]
    lrm_f1.append(f1_score(user_df[y_col], user_df['lrm']))
    gbm_f1.append(f1_score(user_df[y_col], user_df['gbm']))
mean_lrm_f1 = round(np.mean(lrm_f1), 3)
mean_gbm_f1 = round(np.mean(gbm_f1), 3)

print('Per-User Optimized F1 Scores:')
print(f'Linear Regression: {mean_lrm_f1}')
print(f'Gradient Boosting Decision Trees: {mean_gbm_f1}')

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Per-User Optimized F1 Scores:
Linear Regression: 0.367
Gradient Boosting Decision Trees: 0.368


<a id='sec7'></a>
### 7. Finding the Most Influential Features
I charted the feature coefficients for both models to see which predictors most influence the models.

In [305]:
feature_df = pd.DataFrame({'feature': x_cols,
                           'lrm_coef': lrm.coef_[0].tolist(),
                           'lrm_abs_coef': np.abs(lrm.coef_[0]).tolist(),
                           'xgb_coef': gbm.feature_importances_.tolist()})

<a id='sec8'></a>
### 8. Histograms of User F1 Scores
I created histograms of user F1 scores for both models.

In [309]:
pd.DataFrame(gbm_f1).to_csv('tableau/hist.csv')

<a id='sec9'></a>
### 9. Sample User Cart Classification


In [317]:
opt = []
for user in users:
    sdf = score_df[score_df['user_id'] == user]
    if f1_score(sdf[y_col], sdf['gbm_adj']) == .8:
        opt.append(user)

In [319]:
opt[:10]

[165457, 131275, 55976, 149166, 200230, 10673, 33350, 136527, 184394, 40141]

In [326]:
prob_list = gbm.predict_proba(df[df['user_id'] == 55976][x_cols].values)[:,1].tolist()
pid = df[df['user_id'] == 55976]['product_id'].tolist()

In [329]:
p_df = pd.DataFrame({'product_id':pid, 'prob':prob_list})

In [331]:
p_df.head()

,product_id,prob
0,4913,0.036324
1,12341,0.065523
2,12899,0.041584
3,22046,0.642222
4,8859,0.066379


In [332]:
products_df = pd.read_csv('../data/common/products.csv')
products_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [333]:
p_df = p_df.merge(products_df, how='left', on='product_id')

In [336]:
p_df.sort_values(by='prob', ascending=False)

,product_id,prob,product_name,aisle_id,department_id
3,22046,0.642222,Frozen Whole Strawberries,116,1
12,5258,0.205645,Sparkling Water,115,7
5,13176,0.124902,Bag of Organic Bananas,24,4
6,30827,0.101226,Seedless Cucumbers,32,4
10,26348,0.073586,Mixed Fruit Fruit Snacks,50,19
4,8859,0.066379,Natural Spring Water,115,7
1,12341,0.065523,Hass Avocados,32,4
17,27885,0.065380,Malbec,28,5
11,33768,0.049367,Sinfully Sweet Campari Tomatoes,83,4
14,33065,0.048476,Cabernet Sauvignon,28,5
